In [1]:
print('ok')

ok


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import pypdf

In [13]:
def load_pdf(data):
    loader= DirectoryLoader(data,loader_cls=PyPDFLoader,glob='*.pdf')
    
    documents=loader.load()
    return documents

In [15]:
extracted_data=load_pdf('data/')

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [21]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))


5961


In [22]:
def huggingface_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
    return embeddings

In [23]:
embeddings= huggingface_embeddings()

c:\Users\mohit\Documents\Medical-Chatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [26]:
query_result=embeddings.embed_query('Hello world')
print(len(query_result))

384


In [39]:
import os
from dotenv import load_dotenv
load_dotenv()
from pinecone import ServerlessSpec

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [40]:
pc =Pinecone(api_key=PINECONE_API_KEY)

In [42]:
index_name='medical-chatbot'

if not  pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws',region='us-east-1'),
    )

index= pc.Index(index_name)


In [50]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
vector_store=PineconeVectorStore(index=index,embedding=embeddings)\

vector_store.add_documents(text_chunks)

['09d9ae39-cbcb-4422-b4f9-a739a44dd41e',
 '15f628b6-817a-403e-ac9b-b222e966a8b0',
 '32249a12-b1bf-4cf5-b721-4e00e39a9db1',
 '0c2f9fb7-77db-4970-a605-3be7db07c599',
 '5c8ac579-8fc1-4820-9f32-d5965ad25d61',
 '0951b85a-19ec-44dc-8f8b-f5bd785c7a7a',
 'a477947b-49d7-45fa-a12c-6f7915760454',
 'b97bb945-7527-4981-86ff-81157b69be11',
 'c3de75a4-9cbc-4c03-b774-329ef834b50d',
 '7846d091-e5f2-4b55-8ee8-4f7121c59477',
 '4a1c1d4e-1b6a-4cf0-a1ba-7ee663876eeb',
 '57b4977e-da26-4465-9ddc-e7eaf191b2bd',
 'f8cc2535-6675-4e48-ab1d-5ddc3561496b',
 '1accc9bc-6766-4a35-852f-45de528167c0',
 '5460ea53-cadc-4e66-be1a-6cc8c1ad1342',
 '30b0dac0-d628-4989-99df-a4a841029fd7',
 '2b9f4d83-3c21-49d8-8dbf-47c0fac152b3',
 '5a52121c-2b3f-4198-9aa9-ebeb9ad4c639',
 'f8795940-b445-41c6-912c-77aaf048a93a',
 '242149b8-719b-43b8-bfa2-611a78c560d2',
 '1a15c3b4-fb9d-4543-a157-e0631bd50395',
 'f91ce6a8-5367-444d-889c-0ed5f74d9954',
 'db2bb946-30ef-407a-a6d5-e0a675e54aff',
 '5736596d-67d3-4d00-b91a-dd2964cadf23',
 '5653995d-0b6d-

In [55]:
results= vector_store.similarity_search(
    'what is Blood urea nitrogen (BUN)',
    k=2,

)

In [56]:
for res in results:
    print(f"*{res.page_content} [{res.metadata}]")

*Blood urea nitrogen test
Definition
The blood urea nitrogen (BUN) test measures the
level of urea nitrogen in a sample of the patient’s blood.
Urea is a substance that is formed in the liver when the
body breaks down protein. Urea then circulates in the
blood in the form of urea nitrogen. In healthy people,
most urea nitrogen is filtered out by the kidneys and
leaves the body in the urine. If the patient’s kidneys are
not functioning properly or if the body is using large [{'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 548.0, 'page_label': '549', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}]
*are carried out of the body. At the same time, electrolytes
and other chemicals are added to the blood. The purified,
chemically-balanced blood is then returned to the body.
KEY TERMS
Blood urea nitrogen (BUN)—A waste product that
is formed in the l